In [1]:
import numpy as np
from glob import glob
import pandas as pd

files = glob('data/*.csv')
totals = ['Total_PorosityQuantity', 'Total_PorosityQuality', 'Total_UnfilledZones', 'Total_FillingQuality', 'TOTAL_QUALITY']

df_list = list()
target = list()

for i, file in enumerate(files[:100]):
    df = pd.read_csv(file)
    target.append(df.TOTAL_QUALITY.unique()[0])
    df = df.drop(axis=1, columns=totals)  
    df['id'] = i
    
    ts_cols = [col for col in df if col.endswith(('Time', 'id', 'Flow rate', 'Pressure'))]
    df_list.append(df[ts_cols])

df = pd.concat(df_list)
df.head()

AttributeError: module 'pandas' has no attribute 'read_csv'

In [7]:
df.id.unique()

NameError: name 'df' is not defined

In [ ]:
type(df.id.values)

In [ ]:
from tsfresh import extract_features
from tsfresh.utilities.distribution import MultiprocessingDistributor
distributor = MultiprocessingDistributor(n_workers=8, disable_progressbar=False, progressbar_title="Feature Extraction")
extracted_features = extract_features(df, column_id='id', column_sort='Time', distributor=distributor)
extracted_features.to_csv('temp/extracted_features.csv')

### READ EXTRACTED FEATURES FROM CSV

In [ ]:


extracted_features = pd.read_csv('temp/extracted_features.csv')
extracted_features.head()

### GENERATE FIRST MODEL AND TEST

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

extracted_features = extracted_features.dropna(axis=1)
extracted_features = extracted_features.replace([np.inf, -np.inf], np.nan).dropna(axis=1)

X_train, X_test, y_train, y_test = train_test_split(extracted_features, target, test_size=.3)
rf_model = RandomForestClassifier(n_estimators=100).fit(X_train, np.ravel(y_train))
y_pred = rf_model.predict(X_test)

accuracy_score(y_test, y_pred)

In [ ]:
import seaborn as sn
